# eICU Collaborative Research Database

# Workshop 1: Exploring the `patient` table

Documentation on the eICU Collaborative Research Database can be found at: http://eicu-crd.mit.edu/. The `patient` table contains patient demographics and admission and discharge details for hospital and ICU stays. For more detail on the `patient` table, see: http://eicu-crd.mit.edu/eicutables/patient/

In [ ]:
# Import libraries
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import seaborn as sbn

In [ ]:
# Plot settings
%matplotlib inline
plt.style.use('ggplot')
fontsize = 20 # size for x and y ticks
plt.rcParams['legend.fontsize'] = fontsize
plt.rcParams.update({'font.size': fontsize})

In [ ]:
# Database config
sqluser = 'postgres'
dbname = 'eicu'
schema_name = 'eicu_crd'

In [ ]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

## 1. Display list of tables

In [ ]:
query = \
"""
SELECT tablename 
FROM pg_catalog.pg_tables 
WHERE schemaname like 'eicu_crd'
ORDER BY tablename;
"""

list_of_tables = pd.read_sql_query(query,con)

In [ ]:
list_of_tables

## 2. Reviewing the patient table

In [ ]:
# query to load data from the patient table
query = \
"""
SELECT *
FROM patient
"""

print(query)

In [ ]:
# run the query and assign the output to a variable
patient_tab = pd.read_sql_query(query,con)

In [ ]:
# display the first few rows of the dataframe
patient_tab.head()

In [ ]:
# list all of the columns in the table
patient_tab.columns

In [ ]:
# select a limited number of columns to view
columns = ['patientunitstayid','gender','age','unitdischargestatus']
patient_tab[columns].head()

In [ ]:
# what are the unique values for age?
age_col = 'age'
patient_tab[age_col].sort_values().unique()

## Questions

- Try plotting a histogram of ages using the commands in the cell below. Why does the plot fail?

```python
# try plotting a histogram of ages
figsize = (18,8)
patient_tab[age_col].plot(kind='hist',
                          figsize=figsize, 
                          fontsize=fontsize,
                          bins=15)
```

In [ ]:
# create a column containing numerical ages
# If ‘coerce’, then invalid parsing will be set as NaN
agenum_col = 'age_num'
patient_tab[agenum_col] = pd.to_numeric(patient_tab[age_col], errors='coerce')
patient_tab[agenum_col].sort_values().unique()

In [ ]:
# try plotting a histogram of ages
figsize = (18,8)
patient_tab[agenum_col].plot(kind='hist',
                             figsize=figsize, 
                             fontsize=fontsize,
                             bins=15)

In [ ]:
# get some descriptive details about the age of patients
patient_tab[agenum_col].describe()

## Questions

- What is the mean age of patients in this calculation? Why is this lower than the true mean?
- Use the `describe()` method to explore the `admissionweight` of patients. What issue do you see?

In [ ]:
# set threshold based on 99th quantile
adweight_col = 'admissionweight'
quant = patient_tab[adweight_col].quantile(0.99)
patient_tab[patient_tab[adweight_col] > quant] = None

In [ ]:
# describe the admission weights
patient_tab[adweight_col].describe()

## Questions

- What is the average change in weight between `admissionweight` and `dischargeweight`?
- Plot a distribution of the weight change

In [ ]:
# set threshold based on 99th quantile
disweight_col = 'dischargeweight'
quant = patient_tab[disweight_col].quantile(0.99)
patient_tab[patient_tab[disweight_col] > quant] = None

In [ ]:
# describe the discharge weights
patient_tab[disweight_col].describe()

In [ ]:
patient_tab['weight_change'] = patient_tab[adweight_col] - patient_tab[disweight_col]

In [ ]:
# plot the weight changes
figsize = (18,8)
patient_tab['weight_change'].plot(kind='hist',
                             figsize=figsize, 
                             fontsize=fontsize,
                             bins=50)